# <p style="text-align: center;">**Exercise**</p>

## **Aim**: Explore pythons's Scapy module.

### 1.Installation:

In [1]:
pip install --pre scapy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from scapy.all import *

In [4]:
ip = IP()
ip.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = ip
  chksum    = None
  src       = 192.168.1.103
  dst       = 127.0.0.1
  \options   \



### 2. Transmitting packets:

In [5]:
conf.verb=1
conf.color_theme = RastaTheme()


In [6]:
Ether()/IP()/IP()/UDP()

<Ether  type=IPv4 |<IP  frag=0 proto=4 |<IP  frag=0 proto=udp |<UDP  |>>>>

In [7]:
IP(proto=55)/TCP(dport=22)


<IP  frag=0 proto=55 |<TCP  dport=ssh |>>

In [19]:
#Build ARP packets
arpPkt = Ether()/ARP()
arpPkt.show()


###[ Ethernet ]### 
  dst= c8:d7:79:9f:fe:fd
  src= fc:de:56:ff:01:06
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= fc:de:56:ff:01:06
     psrc= 192.168.1.103
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [18]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'
arpPkt.show()


###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= fc:de:56:ff:01:06
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= fc:de:56:ff:01:06
     psrc= 192.168.1.103
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 0.0.0.0



In [17]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt.show()


###[ Ethernet ]### 
  dst= c8:d7:79:9f:fe:fd
  src= fc:de:56:ff:01:06
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= fc:de:56:ff:01:06
     psrc= 192.168.1.103
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 0.0.0.0



In [11]:
sendp(arpPkt)



Sent 1 packets.


In [12]:
#Sending HTTP request
httpPkt = IP(dst="example.com")/TCP(dport=80)/"GET /index.html HTTP/1.1\r\n\r\n"
bytes(httpPkt)

b'E\x00\x00D\x00\x01\x00\x00@\x06\x82\xc9\xc0\xa8\x01g]\xb8\xd8"\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\xd5\xbb\x00\x00GET /index.html HTTP/1.1\r\n\r\n'

In [13]:
hexdump(httpPkt)


0000  45 00 00 44 00 01 00 00 40 06 82 C9 C0 A8 01 67  E..D....@......g
0010  5D B8 D8 22 00 14 00 50 00 00 00 00 00 00 00 00  ].."...P........
0020  50 02 20 00 D5 BB 00 00 47 45 54 20 2F 69 6E 64  P. .....GET /ind
0030  65 78 2E 68 74 6D 6C 20 48 54 54 50 2F 31 2E 31  ex.html HTTP/1.1
0040  0D 0A 0D 0A                                      ....


In [14]:
httpPkt.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= tcp
  chksum= None
  src= 192.168.1.103
  dst= Net('example.com')
  \options\
###[ TCP ]### 
     sport= ftp_data
     dport= http
     seq= 0
     ack= 0
     dataofs= None
     reserved= 0
     flags= S
     window= 8192
     chksum= None
     urgptr= 0
     options= []
###[ Raw ]### 
        load= 'GET /index.html HTTP/1.1\r\n\r\n'



In [15]:
#TCP/IP full three-way handshake
#send SYN packet
syn = IP(dst='google.com')/TCP(dport=80, flags='S')
syn

<IP  frag=0 proto=tcp dst=Net('google.com') |<TCP  dport=http flags=S |>>

In [16]:
# Receive SYN-ACK packet from google.com
syn_ack = sr1(syn)


Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


### 3. Sniffing packets:

In [20]:
pkts = sniff(count=2)
pkts

<Sniffed: TCP:0 UDP:2 ICMP:0 Other:0>

In [21]:
#Keeping track of number of packets sniffed
from collections import Counter
from scapy.all import sniff
packet_counts = Counter()
def custom_action(packet):
    key = tuple(sorted([packet[0][1].src, packet[0][1].dst]))
    packet_counts.update([key])
    return f"Packet #{sum(packet_counts.values())}: {packet[0][1].src} ==> {packet[0][1].dst}"
sniff(filter="ip", prn=custom_action, count=10)
print("\n".join(f"{f'{key[0]} <--> {key[1]}'}: {count}" for key, count in packet_counts.items()))

Packet #1: 52.109.12.20 ==> 192.168.1.103
Packet #2: 192.168.1.103 ==> 52.109.12.20
Packet #3: 192.168.1.103 ==> 239.255.255.250
Packet #4: 192.168.1.103 ==> 239.255.255.250
Packet #5: 192.168.1.103 ==> 13.227.178.25
Packet #6: 13.227.178.25 ==> 192.168.1.103
Packet #7: 192.168.1.103 ==> 13.227.178.25
Packet #8: 192.168.1.103 ==> 13.227.178.25
Packet #9: 13.227.178.25 ==> 192.168.1.103
Packet #10: 13.227.178.25 ==> 192.168.1.103
192.168.1.103 <--> 52.109.12.20: 2
192.168.1.103 <--> 239.255.255.250: 2
13.227.178.25 <--> 192.168.1.103: 6


### 4. Traceroute:

In [22]:
help(traceroute)


Help on function traceroute in module scapy.layers.inet:

traceroute(target, dport=80, minttl=1, maxttl=30, sport=<RandShort>, l4=None, filter=None, timeout=2, verbose=None, **kargs)
    Instant TCP traceroute
    
    :param target:  hostnames or IP addresses
    :param dport:   TCP destination port (default is 80)
    :param minttl:  minimum TTL (default is 1)
    :param maxttl:  maximum TTL (default is 30)
    :param sport:   TCP source port (default is random)
    :param l4:      use a Scapy packet instead of TCP
    :param filter:  BPF filter applied to received packets
    :param timeout: time to wait for answers (default is 2s)
    :param verbose: detailed output
    :return: an TracerouteResult, and a list of unanswered packets



In [23]:
conf.route


Network          Netmask          Gateway      Iface                                       Output IP        Metric
0.0.0.0          0.0.0.0          192.168.1.1  Remote NDIS based Internet Sharing Device   192.168.1.103    25
192.168.0.0      255.255.0.0      0.0.0.0      Remote NDIS based Internet Sharing Device   192.168.1.103    281
192.168.1.103    255.255.255.255  0.0.0.0      Remote NDIS based Internet Sharing Device   192.168.1.103    281
192.168.255.255  255.255.255.255  0.0.0.0      Remote NDIS based Internet Sharing Device   192.168.1.103    281
224.0.0.0        255.0.0.0        0.0.0.0      Bluetooth Device (Personal Area Network)    169.254.2.122    321
224.0.0.0        255.0.0.0        0.0.0.0      Intel(R) Wireless-N 7265                    169.254.21.30    281
224.0.0.0        255.0.0.0        0.0.0.0      Microsoft Wi-Fi Direct Virtual Adapter      169.254.181.83   281
224.0.0.0        255.0.0.0        0.0.0.0      Microsoft Wi-Fi Direct Virtual Adapter #3   169.254.24.

In [24]:
traceroute(target="www.google.com", maxttl=20)
# Result shows TCP traceroute on default port 80 with (SA - Syn Ack flags)


Begin emission:
Finished sending 20 packets.

Received 21 packets, got 14 answers, remaining 6 packets
   216.58.196.68:tcp80 
1  192.168.1.1     11  
3  10.72.59.50     11  
4  192.168.37.150  11  
5  192.168.37.205  11  
11 74.125.32.32    11  
12 108.170.229.197 11  
13 209.85.255.209  11  
14 216.58.196.68   SA  
15 216.58.196.68   SA  
16 216.58.196.68   SA  
17 216.58.196.68   SA  
18 216.58.196.68   SA  
19 216.58.196.68   SA  
20 216.58.196.68   SA  


(<Traceroute: TCP:7 UDP:0 ICMP:7 Other:0>,
 <Unanswered: TCP:6 UDP:0 ICMP:0 Other:0>)